# Relationale Algebra

## Das Formale Konzept hinter Anfragesprachen, wie z.B. SQL

In der Vorlesung haben wir die Operatoren kennengelernt.
Dieses Notebook deckt nicht alle Operatoren der Relationalen Algebra ab. Für einen Überblick bitte das Vorlesungsmaterial betrachten!
Der Zweck dieses Notebooks ist Relationale Algebra interaktiv zu üben.
Dafür benutzen wir eine erweiterte Version der Python Library [reframe](https://github.com/bnmnetp/reframe).

Los gehts!
Zuerst definieren wir eine Relation und benutzen als Eingabe die beigefügte CSV Datei.
Relationale Algebra Ausdrücke haben als Eingabe eine (oder mehrere) Relation(en), und gibt eine neue Relation als Ausgabe zurück.

In [ ]:
from reframe import Relation
country = Relation('../resources/04_relationale_algebra/country.csv')
country

Die Operation σ (Selektion) begrenzt die Tupel der Eingaberelation, anhand des gegebenens Prädikats.
Wir wollen jetzt die Länder ausgeben lassen, welche sich in Asien oder Europa befinden.

![Select Example](../resources/04_relationale_algebra/select_example.png)

In [ ]:
country.select('continent=="Asia" | continent=="Europe"')

Die Operation π (Projektion) begrenzt die Attribute der Eingaberelation, anhand der gegebenen Attributnamen.
In unserem Beispiel, wollen wir nun nur den Ländercode, Namen und Kontinent der Länder die sich entweder in Asien oder Europa befinden ausgeben lassen.

![select_project](../resources/04_relationale_algebra/select_project.png)

In [ ]:
country.select('continent=="Asia" | continent=="Europe"').project(["code","name", "continent"])

Die Reihenfolge dieser Operationen spielt keine Rolle, folgendes liefert also dasselbe Ergebnis mit dem oben ausgeführtem Ausdruck:

![project_select](../resources/04_relationale_algebra/project_select.png)

In [ ]:
country.project(["code","name", "continent"]).select('continent=="Asia" | continent=="Europe"')

Der Operator ρ erlaubt es, ein Attribut in der Ausgangsrelation umbzubenennen

![select_project_rename](../resources/04_relationale_algebra/select_project_rename.png)

In [ ]:
country.select('continent=="Asia" | continent=="Europe"').project(["code","name", "continent"]).rename("name","Land")


Bisher haben wir unäre Operatoren betrachtet. Jetzt betrachten wir binäre Operationen.
Der Operator - erlaubt es, die Differenz von zwei Relationen zu berechnen. Folgender Ausdruck gibt alle Länder die nicht zu Asien gehören zurück.

![cut](../resources/04_relationale_algebra/select_diff.png)

In [ ]:
asia = country.select('continent=="Asia"')

country.minus(asia)

Der Operator ∪ erlaubt es, zwei Relationen zu vereinigen. Folgender Ausdruck gibt die Länder die zu Europa und Asien gehören zurück.

![union](../resources/04_relationale_algebra/select_union.png)

In [ ]:
asia = country.select('continent=="Asia"')
europe = country.select('continent=="Europe"')

asia.union(europe)


Mit dem Operator ⨯ wird das kartesische Produkt gebildet.
Wir definieren eine neue Relation country_stats, welche Statistiken beinhaltet und bilden das kartesische Produkt über country und country_stats.

![cartesian](../resources/04_relationale_algebra/cartesian.png)

In [ ]:
country_stats = Relation('../resources/04_relationale_algebra/country_stats.csv')
country.cartesian_product(country_stats)


Der Natural Join Operator hat zwei Relationen als Eingaben und gibt eine neue Relation mit diesen Tupeln zurück, bei denen die Werte der gemeinsamen Attribute übereinstimmen.
Zum Beispiel, wenn wir die Relationen

![njoin](../resources/04_relationale_algebra/njoin.png)

In [ ]:
country.njoin(country_stats)

θ-join ist ein erweiterter Operator, der das kartesische Produkt und eine Selektion kombiniert um zwei Relationen zu joinen. Im folgenden, sehen wir wie der θ-join Operator aus den Basisoperationen abgeleitet werden kann.

![tjoin](../resources/04_relationale_algebra/tjoin.png)

In folgendem Ausdruck wird ein kartesisches Produkt mit einer Selektion benutzt und wir erhalten das selbe Ergebnis wie oben.


In [ ]:
country.cartesian_product(country_stats.rename("code","countrycode")).select("code == countrycode").project(["code","name","continent","region","surfacearea","indepyear","population","lifeexpectancy","gnp","gnpold","localname","governmentform","headofstate","capital","code2"])


Der intersection Operator erlaubt es die Schnittmenge zweier Relationen zu bilden.
Wir führen zwei neue Relationen ein. Eine die Länder mit hoher Lebenserwartung entält, und eine andere die Länder mit großer Bevölkerung enthält.
Wir wollen die Schnittmenge dieser Relationen berechnen:

![select_cut](../resources/04_relationale_algebra/select_cut.png)

In [ ]:
highlifeexp = country_stats.select('lifeexpectancy > 60')
largepopulation = country_stats.select('population > 80000000')

highlifeexp.intersect(largepopulation)


Ungruppierte Aggregationen erlauben es Aggregate wie Minimum, Maximum und Durschnitt auf Relationen zu berechnen.
Im nächsten Beispiel wollen wir die Anzahl der Tupel unserer Relation berechnen:

In [ ]:
country.count()

Guppierte Aggregationen erlauben es Aggregate wie Minimum, Maximum und Durschnitt auf Gruppen zu berechnen.
Im nächsten Beispiel berechnen wir die Anzahl der Länder in jedem Kontinent.

In [ ]:
country.groupby('continent').count('code')

Im nächsten Beispiel wollen wir den Ländercode und die Fläche des Landes mit der kleinsten Fläche berechnen:

In [ ]:
country.njoin(country_stats).min('surfacearea').cartesian_product(country_stats).select("min==surfacearea").project(["code","surfacearea"])